# LLM RAG Accuracy Comparison

This notebook compares multiple LLM models on RAG (Retrieval-Augmented Generation) accuracy using:
- **Embedding**: Gemini `text-embedding-004` (768 dimensions)
- **Vector Store**: MongoDB Atlas with cosine similarity
- **Fact Table**: 20 Q&A pairs from UEL (University of Economics and Law) documents

## Prerequisites
Before running this notebook, ensure you have:
1. Run `scripts/embed_pdfs_to_mongo.py` to embed PDFs into MongoDB
2. Created a vector search index named `vector_index` in MongoDB Atlas
3. Set up `.env` with `GEMINI_API_KEY`, `GROQ_API_KEY`, `MONGODB_URI2`, `MONGODB_DB_NAME2`

In [48]:
# Install required packages if needed
# !pip install google-genai groq langchain-mongodb langchain-google-genai pymongo python-dotenv pandas tabulate

In [49]:
import os
import json
import re
from dotenv import load_dotenv
from pymongo import MongoClient
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_mongodb import MongoDBAtlasVectorSearch
import pandas as pd
from typing import List, Dict
import time

# Load environment variables
load_dotenv()

# Configuration
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
MONGODB_URI = os.getenv("MONGODB_URI2")
MONGODB_DB_NAME = os.getenv("MONGODB_DB_NAME2")
COLLECTION_NAME = "pdf"
INDEX_NAME = "vector_index"

print("Environment loaded successfully!")
print(f"MongoDB DB: {MONGODB_DB_NAME}")

Environment loaded successfully!
MongoDB DB: thesis


In [50]:
# Load fact table
import os

# Get the absolute path to the fact table (same directory as notebook)
SCRIPT_DIR = os.path.dirname(os.path.abspath(__file__)) if '__file__' in dir() else os.getcwd()
FACT_TABLE_PATH = os.path.join(SCRIPT_DIR, "fact_table.json")

# Fallback paths
if not os.path.exists(FACT_TABLE_PATH):
    FACT_TABLE_PATH = "fact_table.json"  # Same directory
if not os.path.exists(FACT_TABLE_PATH):
    FACT_TABLE_PATH = "scripts/fact_table.json"  # From project root
if not os.path.exists(FACT_TABLE_PATH):
    FACT_TABLE_PATH = r"c:\Projects\Thesis-RAG-Langgraph-Assistant\scripts\fact_table.json"  # Absolute path

print(f"Loading from: {FACT_TABLE_PATH}")

with open(FACT_TABLE_PATH, 'r', encoding='utf-8') as f:
    fact_table = json.load(f)

questions_and_answers = fact_table["questions_and_answers"]
print(f"Loaded {len(questions_and_answers)} questions from fact table")

# Display sample
for qa in questions_and_answers[:3]:
    print(f"\nQ{qa['id']}: {qa['question'][:80]}...")
    print(f"A: {qa['answer'][:100]}...")

Loading from: c:\Projects\Thesis-RAG-Langgraph-Assistant\scripts\fact_table.json
Loaded 10 questions from fact table

Q1: Thư viện UELlib có bao nhiêu bản tài liệu (cập nhật 30/6/2025)?...
A: Thư viện UELlib có nguồn tài nguyên thông tin đa dạng, với 66.419 bản tài liệu (cập nhật 30/6/2025)....

Q2: Địa chỉ của Thư viện Trường Đại học Kinh tế - Luật là gì?...
A: Địa chỉ: Số 669 Đỗ Mười, Khu phố 13, phường Linh Xuân, TP. HCM. Thư viện đặt tại tòa nhà KTL B2 (tần...

Q3: Năm học 2025-2026, UEL có bao nhiêu ngành/chuyên ngành đào tạo đại học?...
A: Năm học 2025-2026, UEL có 35 ngành/chuyên ngành đào tạo đại học, trong đó 24 chương trình dạy và học...


In [ ]:
# Setup MongoDB Vector Store for RAG
client = MongoClient(MONGODB_URI)
collection = client[MONGODB_DB_NAME][COLLECTION_NAME]

# Initialize embeddings
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    google_api_key=GEMINI_API_KEY
)

# Create vector store
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name=INDEX_NAME,
    relevance_score_fn="cosine",
    text_key="text"  # Match the field name used in embed_pdfs_to_mongo.py
)

print("Vector store initialized!")
print(f"Documents in collection: {collection.count_documents({})}")

Vector store initialized!
Documents in collection: 403


In [52]:
def retrieve_context(question: str, k: int = 5) -> str:
    """Retrieve relevant context from vector store."""
    docs = vector_store.similarity_search(question, k=k)
    context = "\n\n".join([doc.page_content for doc in docs])
    return context

# Test retrieval
test_context = retrieve_context(questions_and_answers[0]["question"])
print(f"Retrieved context length: {len(test_context)} characters")
print(f"Preview: {test_context[:500]}...")

Retrieved context length: 3787 characters
Preview: tĩnh,

phòng

học

nhóm Thư viện UELlib có nguồn tài nguyên thông tin đa dạng, với 66.419 bản tài liệu (cập nhật 30/6/2025) trải

đều

ở

các

ngành

học

cùng

với

các

lĩnh

vực

khác,

hàng

trăm

triệu

tài

nguyên

thông

tin

điện

tử

gồm

bài

báo

nghiên

cứu,

sách,

tạp

chí,

luận

văn-luận

án

và

nhiều

tài

liệu

khoa

học

khác

được

tập

hợp

từ

các

Nhà

xuất

bản

uy

tín

trên

thế

giới

và

trong

nước.

Trong

đó

bao

gồm

kho

thông

tin

khoa

học

số

của

các

trư...


In [55]:
# Initialize LLM clients
from google import genai
from groq import Groq

# Gemini client
gemini_client = genai.Client(api_key=GEMINI_API_KEY)

# Groq client
groq_client = Groq(api_key=GROQ_API_KEY)

print("LLM clients initialized!")

LLM clients initialized!


In [ ]:
# Define model configurations
MODELS = {
    "Gemini 2.5 Flash": {
        "provider": "gemini",
        "model_id": "gemini-2.5-flash"
    },
    "Gemini 2.5 Pro": {
        "provider": "gemini",
        "model_id": "gemini-2.5-pro"
    },
    "Llama 3.3 70B": {
        "provider": "groq",
        "model_id": "llama-3.3-70b-versatile"
    },
    "GPT-OSS 120B": {
        "provider": "groq",
        "model_id": "openai/gpt-oss-120b"
    },
    "Kimi K2": {
        "provider": "groq",
        "model_id": "moonshotai/kimi-k2-instruct-0905"
    }
}

print(f"Configured {len(MODELS)} models for testing")

Configured 6 models for testing


In [57]:
def create_rag_prompt(question: str, context: str) -> str:
    """Create a RAG prompt with context and question."""
    return f"""Dựa vào ngữ cảnh được cung cấp dưới đây, hãy trả lời câu hỏi một cách chính xác và ngắn gọn.
Chỉ sử dụng thông tin từ ngữ cảnh để trả lời. Nếu không tìm thấy thông tin, hãy nói "Không tìm thấy thông tin".

NGỮ CẢNH:
{context}

CÂU HỎI: {question}

TRẢ LỜI:"""


def query_gemini(prompt: str, model_id: str) -> str:
    """Query Gemini model."""
    try:
        response = gemini_client.models.generate_content(
            model=model_id,
            contents=prompt
        )
        return response.text
    except Exception as e:
        return f"Error: {str(e)}"


def query_groq(prompt: str, model_id: str) -> str:
    """Query Groq model."""
    try:
        response = groq_client.chat.completions.create(
            model=model_id,
            messages=[{"role": "user", "content": prompt}],
            temperature=0.1,
            max_tokens=1024
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {str(e)}"


def query_model(question: str, context: str, model_name: str) -> str:
    """Query a model with RAG context."""
    model_config = MODELS[model_name]
    prompt = create_rag_prompt(question, context)
    
    if model_config["provider"] == "gemini":
        return query_gemini(prompt, model_config["model_id"])
    else:  # groq
        return query_groq(prompt, model_config["model_id"])

print("Query functions defined!")

Query functions defined!


In [58]:
def normalize_text(text: str) -> str:
    """Normalize text for comparison."""
    # Convert to lowercase
    text = text.lower()
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Remove punctuation at end
    text = re.sub(r'[.,:;!?]+$', '', text)
    return text


def extract_key_facts(answer: str) -> List[str]:
    """Extract key facts from an answer (numbers, names, dates)."""
    facts = []
    
    # Extract numbers (including decimals and percentages)
    numbers = re.findall(r'\d+[.,]?\d*%?', answer)
    facts.extend(numbers)
    
    # Extract Vietnamese proper nouns (capitalized words)
    # Look for patterns like "PGS.TS", "ThS.", names, etc.
    names = re.findall(r'(?:PGS\.|TS\.|ThS\.|GS\.)?\s*[A-ZÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ][a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]*(?:\s+[A-ZÀÁẠẢÃÂẦẤẬẨẪĂẰẮẶẲẴÈÉẸẺẼÊỀẾỆỂỄÌÍỊỈĨÒÓỌỎÕÔỒỐỘỔỖƠỜỚỢỞỠÙÚỤỦŨƯỪỨỰỬỮỲÝỴỶỸĐ][a-zàáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ]*)+', answer)
    facts.extend([n.strip() for n in names if len(n.strip()) > 3])
    
    return facts


def calculate_accuracy(model_answer: str, ground_truth: str) -> float:
    """Calculate accuracy score between model answer and ground truth."""
    if not model_answer or "Error" in model_answer or "Không tìm thấy" in model_answer:
        return 0.0
    
    # Extract key facts from both
    gt_facts = extract_key_facts(ground_truth)
    model_facts = extract_key_facts(model_answer)
    
    if not gt_facts:
        # If no key facts, use text similarity
        gt_normalized = normalize_text(ground_truth)
        model_normalized = normalize_text(model_answer)
        
        # Check if key phrases from ground truth appear in model answer
        gt_words = set(gt_normalized.split())
        model_words = set(model_normalized.split())
        
        if not gt_words:
            return 0.0
        
        overlap = len(gt_words.intersection(model_words))
        return min(overlap / len(gt_words), 1.0)
    
    # Count how many ground truth facts appear in model answer
    matches = 0
    model_answer_lower = model_answer.lower()
    
    for fact in gt_facts:
        if fact.lower() in model_answer_lower:
            matches += 1
    
    return matches / len(gt_facts) if gt_facts else 0.0


# Test accuracy calculation
test_gt = "Trường có 428 nhân sự, trong đó 48,5% có trình độ TS, PGS và GS"
test_model = "Theo thông tin, trường có 428 nhân sự với 48,5% có trình độ Tiến sĩ trở lên."
print(f"Test accuracy: {calculate_accuracy(test_model, test_gt):.2%}")

Test accuracy: 100.00%


In [66]:
# Run evaluation on all models
results = {model_name: [] for model_name in MODELS.keys()}
detailed_results = []

# Limit questions for testing (remove this line to run all 20)
test_questions = questions_and_answers  # Use all 20 questions

print(f"Running evaluation on {len(test_questions)} questions...")
print("=" * 60)

for i, qa in enumerate(test_questions):
    question = qa["question"]
    ground_truth = qa["answer"]
    q_id = qa["id"]
    
    print(f"\nQuestion {q_id}/{len(test_questions)}: {question[:60]}...")
    
    # Retrieve context once for all models
    context = retrieve_context(question)
    
    for model_name in MODELS.keys():
        print(f"  - Querying {model_name}...", end=" ")
        
        try:
            # Query model
            model_answer = query_model(question, context, model_name)
            
            # Calculate accuracy
            accuracy = calculate_accuracy(model_answer, ground_truth)
            results[model_name].append(accuracy)
            
            # Store detailed result
            detailed_results.append({
                "question_id": q_id,
                "question": question,
                "ground_truth": ground_truth,
                "model": model_name,
                "model_answer": model_answer,
                "accuracy": accuracy
            })
            
            print(f"Accuracy: {accuracy:.2%}")
            
            # Rate limiting
            time.sleep(0.5)
            
        except Exception as e:
            print(f"Error: {e}")
            results[model_name].append(0.0)
            detailed_results.append({
                "question_id": q_id,
                "question": question,
                "ground_truth": ground_truth,
                "model": model_name,
                "model_answer": f"Error: {e}",
                "accuracy": 0.0
            })

print("\n" + "=" * 60)
print("Evaluation complete!")

Running evaluation on 10 questions...

Question 1/10: Thư viện UELlib có bao nhiêu bản tài liệu (cập nhật 30/6/202...
  - Querying Gemini 2.5 Flash... Accuracy: 100.00%
  - Querying Gemini 2.5 Pro... Accuracy: 100.00%
  - Querying Llama 3.3 70B... Accuracy: 100.00%
  - Querying GPT-OSS 120B... Accuracy: 50.00%
  - Querying Qwen3 32B... Accuracy: 100.00%
  - Querying Kimi K2... Accuracy: 50.00%

Question 2/10: Địa chỉ của Thư viện Trường Đại học Kinh tế - Luật là gì?...
  - Querying Gemini 2.5 Flash... Accuracy: 57.14%
  - Querying Gemini 2.5 Pro... Accuracy: 57.14%
  - Querying Llama 3.3 70B... Accuracy: 57.14%
  - Querying GPT-OSS 120B... Accuracy: 28.57%
  - Querying Qwen3 32B... Accuracy: 85.71%
  - Querying Kimi K2... Accuracy: 57.14%

Question 3/10: Năm học 2025-2026, UEL có bao nhiêu ngành/chuyên ngành đào t...
  - Querying Gemini 2.5 Flash... Accuracy: 66.67%
  - Querying Gemini 2.5 Pro... Accuracy: 66.67%
  - Querying Llama 3.3 70B... Accuracy: 66.67%
  - Querying GPT-OSS 120B.

In [67]:
# Calculate summary statistics
summary_data = []

for model_name, accuracies in results.items():
    if accuracies:
        avg_accuracy = sum(accuracies) / len(accuracies)
        max_accuracy = max(accuracies)
        min_accuracy = min(accuracies)
        perfect_scores = sum(1 for a in accuracies if a >= 0.9)
    else:
        avg_accuracy = max_accuracy = min_accuracy = 0.0
        perfect_scores = 0
    
    summary_data.append({
        "Model": model_name,
        "Average Accuracy": f"{avg_accuracy:.1%}",
        "Max Accuracy": f"{max_accuracy:.1%}",
        "Min Accuracy": f"{min_accuracy:.1%}",
        "Perfect Scores (≥90%)": perfect_scores,
        "Questions Tested": len(accuracies)
    })

# Create summary DataFrame
summary_df = pd.DataFrame(summary_data)
summary_df = summary_df.sort_values("Average Accuracy", ascending=False)

print("\n" + "=" * 80)
print("MODEL ACCURACY COMPARISON")
print("=" * 80)
print(summary_df.to_string(index=False))


MODEL ACCURACY COMPARISON
           Model Average Accuracy Max Accuracy Min Accuracy  Perfect Scores (≥90%)  Questions Tested
       Qwen3 32B            93.7%       100.0%        75.0%                      7                10
Gemini 2.5 Flash            89.9%       100.0%        57.1%                      7                10
   Llama 3.3 70B            83.0%       100.0%        50.0%                      5                10
  Gemini 2.5 Pro            81.1%       100.0%        50.0%                      5                10
         Kimi K2            65.4%       100.0%         4.8%                      4                10
    GPT-OSS 120B            47.4%       100.0%         0.0%                      2                10


In [69]:
# Create accuracy by question visualization
pivot_data = []
for dr in detailed_results:
    pivot_data.append({
        "Question": f"Q{dr['question_id']}",
        "Model": dr['model'],
        "Accuracy": dr['accuracy']
    })

pivot_df = pd.DataFrame(pivot_data)
pivot_table = pivot_df.pivot(index="Question", columns="Model", values="Accuracy")

# Try to display with background gradient, fallback to plain table
try:
    import matplotlib.pyplot as plt
    styled_pivot = pivot_table.style.background_gradient(cmap='RdYlGn', vmin=0, vmax=1)\
        .format("{:.0%}")\
        .set_caption("Accuracy by Question and Model")
    display(styled_pivot)
except ImportError:
    # Fallback: display as formatted percentage table
    print("Accuracy by Question and Model")
    print("=" * 80)
    formatted_table = pivot_table.applymap(lambda x: f"{x:.0%}" if pd.notna(x) else "N/A")
    display(formatted_table)

Model,GPT-OSS 120B,Gemini 2.5 Flash,Gemini 2.5 Pro,Kimi K2,Llama 3.3 70B,Qwen3 32B
Question,,,,,,
Q1,50%,100%,100%,50%,100%,100%
Q10,100%,100%,100%,100%,100%,100%
Q2,29%,57%,57%,57%,57%,86%
Q3,17%,67%,67%,17%,67%,100%
Q4,25%,75%,75%,75%,75%,75%
Q5,29%,100%,62%,5%,81%,76%
Q6,100%,100%,100%,100%,100%,100%
Q7,0%,100%,100%,100%,100%,100%
Q8,75%,100%,100%,100%,100%,100%


In [62]:
# Save detailed results to JSON
output_results = {
    "evaluation_date": "2025-12-31",
    "total_questions": len(test_questions),
    "models_tested": list(MODELS.keys()),
    "summary": summary_df.to_dict('records'),
    "detailed_results": detailed_results
}

# Use absolute path for saving (same directory as notebook)
output_path = r"c:\Projects\Thesis-RAG-Langgraph-Assistant\scripts\evaluation_results.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(output_results, f, ensure_ascii=False, indent=2)

print(f"Results saved to {output_path}")

Results saved to c:\Projects\Thesis-RAG-Langgraph-Assistant\scripts\evaluation_results.json


In [70]:
# Show sample detailed comparisons
print("\n" + "=" * 80)
print("SAMPLE DETAILED COMPARISONS")
print("=" * 80)

# Show first 3 questions with all model answers
for q_id in range(1, 4):
    q_results = [dr for dr in detailed_results if dr['question_id'] == q_id]
    if q_results:
        print(f"\n{'='*60}")
        print(f"Question {q_id}: {q_results[0]['question']}")
        print(f"\n✅ Ground Truth: {q_results[0]['ground_truth']}")
        print(f"\n{'─'*60}")
        
        for qr in q_results:
            acc_emoji = "🟢" if qr['accuracy'] >= 0.7 else "🟡" if qr['accuracy'] >= 0.4 else "🔴"
            print(f"\n{acc_emoji} {qr['model']} ({qr['accuracy']:.0%}):")
            print(f"   {qr['model_answer'][:200]}..." if len(qr['model_answer']) > 200 else f"   {qr['model_answer']}")


SAMPLE DETAILED COMPARISONS

Question 1: Thư viện UELlib có bao nhiêu bản tài liệu (cập nhật 30/6/2025)?

✅ Ground Truth: Thư viện UELlib có nguồn tài nguyên thông tin đa dạng, với 66.419 bản tài liệu (cập nhật 30/6/2025).

────────────────────────────────────────────────────────────

🟢 Gemini 2.5 Flash (100%):
   Thư viện UELlib có 66.419 bản tài liệu (cập nhật 30/6/2025).

🟢 Gemini 2.5 Pro (100%):
   Thư viện UELlib có 66.419 bản tài liệu (cập nhật 30/6/2025).

🟢 Llama 3.3 70B (100%):
   Thư viện UELlib có 66.419 bản tài liệu (cập nhật 30/6/2025).

🟡 GPT-OSS 120B (50%):
   66.419 bản tài liệu.

🟢 Qwen3 32B (100%):
   <think>
Okay, let's see. The user is asking how many documents the UELlib library has as of June 30, 2025. I need to check the context provided.

Looking through the text, there's a mention of "66.419...

🟡 Kimi K2 (50%):
   66.419 bản tài liệu.

Question 2: Địa chỉ của Thư viện Trường Đại học Kinh tế - Luật là gì?

✅ Ground Truth: Địa chỉ: Số 669 Đỗ Mười, Khu phố 13, p

In [71]:
# Final summary
print("\n" + "="*80)
print("FINAL SUMMARY")
print("="*80)

best_model = summary_df.iloc[0]
print(f"\n🏆 Best Performing Model: {best_model['Model']}")
print(f"   Average Accuracy: {best_model['Average Accuracy']}")
print(f"   Perfect Scores: {best_model['Perfect Scores (≥90%)']} out of {best_model['Questions Tested']}")

print("\n📊 Model Rankings:")
for idx, (_, row) in enumerate(summary_df.iterrows(), 1):
    print(f"   {idx}. {row['Model']}: {row['Average Accuracy']}")

print("\n" + "="*80)


FINAL SUMMARY

🏆 Best Performing Model: Qwen3 32B
   Average Accuracy: 93.7%
   Perfect Scores: 7 out of 10

📊 Model Rankings:
   1. Qwen3 32B: 93.7%
   2. Gemini 2.5 Flash: 89.9%
   3. Llama 3.3 70B: 83.0%
   4. Gemini 2.5 Pro: 81.1%
   5. Kimi K2: 65.4%
   6. GPT-OSS 120B: 47.4%

